In [1]:
import os
from configparser import ConfigParser

import numpy as np

In [2]:
# Range of tables in wavelengths
wavelength_ranges = np.array([ [2295, 2400] ]) # nm

# Instrument resolution in wl (nm)
# Only needed if computing wavenumber spacing automatically
#wl_resolution = 0.25 # nm

# Override automatic determination of wavenumber spacing if specified
wn_spacing = [ 0.01 ] # cm^-1

# desired species
molecules = ["CH4", "CO", "H2O"]

config_output_directory = "/tmp/absco_build/tropomi"

----

In [3]:
source_config_filename = os.path.realpath(os.path.join("..", "ABSCO_config.ini"))

print("Using source config file:", source_config_filename)

Using source config file: /tb/sandbox2/mcduffie/refractor/absco/ABSCO_config.ini


## Compute Ranges and Spacing in Wavenumbers

In [4]:
# convert to cm^-1
wavenumber_ranges = 1e7 / wavelength_ranges[::-1, ::-1]

# Create wavenumber ranges but rounded down and up at the ends
print("Rounding wavenumber ranges:")
full_spectral_ranges = []
for beg_wn, end_wn in wavenumber_ranges:
    print(f"({beg_wn:.2f}, {end_wn:.2f}) cm^-1 -> ({np.floor(beg_wn)}, {np.ceil(end_wn)}) cm^-1")
    full_spectral_ranges.append((np.floor(beg_wn), np.ceil(end_wn)))
full_spectral_ranges = np.array(full_spectral_ranges)

Rounding wavenumber ranges:
(4166.67, 4357.30) cm^-1 -> (4166.0, 4358.0) cm^-1


In [5]:
# Compute spacing at all points +/- spacing to figure out a good average resolution in cm^-1
def determine_wn_spacing(wl_ranges, wl_res):
    wn_spacing = []
    
    for wl_beg, wl_end in wl_ranges:
        # Sample at twice the resolution
        wl_grid = np.arange(wl_beg, wl_end, wl_res*2)
        wn_diff = 1e7/wl_grid[:-1] - 1e7/wl_grid[1:]
        
        band_spacing = np.mean(wn_diff) * 1e-2
        
        decimal_places = int(f'{band_spacing:e}'.split('e')[-1])

        # Round up to next decimal place
        wn_spacing.append(np.round(band_spacing, -decimal_places-1))

    return wn_spacing

In [6]:
if not "wn_spacing" in locals():
    high_res_spacing = determine_wn_spacing(wavelength_ranges, wl_resolution)
    print(f"Determined a wavenumber spacing of {wn_spacing} from wavelength resolution {wl_resolution:.4f}")
else:
    print(f"Using predefined wavenumber spacing: {wn_spacing}")
    high_res_spacing = np.array(wn_spacing)

Using predefined wavenumber spacing: [0.01]


In [7]:
# Derermine resolution for LBLRTM computations
# Want a lblres ~ 1.5e-4 or smaller
# Must be a power of 2 relationship with output high resolution spacing
power = np.ceil(np.log2(high_res_spacing / 1.5e-4)).astype(int)

lblres = high_res_spacing / 2**power
print("LBLRTM Compuation resolution:", ['%0.10e' % r for r in lblres])

LBLRTM Compuation resolution: ['7.8125000000e-05']


In [8]:
# Check that by converting the resolutions to string that when reinterpreted by the config
# the same power of 2 relationship holds
# Ratio must be a power of 2, check that string formatting is sufficient# 
hr_format = "{:0.10e}"
lr_format = "{:0.10e}"

hr_back_convert = np.array(['%0.10e' % r for r in high_res_spacing]).astype(float)
lr_back_convert = np.array(['%0.10e' % r for r in lblres]).astype(float)
calc_power = np.log2(hr_back_convert / lr_back_convert)

for op, cp in zip(power, calc_power):
    assert(op == cp)

## Create Modified Config

In [9]:
# Read ABSCO configuration from repository as template
# It will have the paths to the ABSCO executables filled in after the build_models.py script has been run
config = ConfigParser()
config.read(source_config_filename);

In [10]:
# Break spectral ranges into beginning and ending portions for config writing
wn1 = []
wn2 = []
for i, (beg, end) in enumerate(full_spectral_ranges):
    wn1.append(beg)
    wn2.append(end)

In [11]:
# Modify config values converting our determined numbers into strings properly representing the numbers chosen
config['channels']['wn1'] = " ".join([ "{:.4f}".format(v) for v in wn1 ])
config['channels']['wn2'] = " ".join([ "{:.4f}".format(v) for v in wn2 ])
config['channels']['lblres'] = " ".join([ hr_format.format(r) for r in lblres])
config['channels']['outres'] = " ".join([ lr_format.format(r) for r in high_res_spacing])

config['molecules']['molnames'] = " ".join([ m.lower() for m in molecules])

In [12]:
# Print out what has been modified in the new config file
print("Modifications made to ABSCO configuration:\n")
print("wn1 =", config['channels']['wn1'])
print("wn2 =", config['channels']['wn2'])
print("lblres =", config['channels']['lblres'])
print("outres =", config['channels']['outres'])
print("molecules =", config['molecules']['molnames'])

Modifications made to ABSCO configuration:

wn1 = 4166.0000
wn2 = 4358.0000
lblres = 7.8125000000e-05
outres = 1.0000000000e-02
molecules = ch4 co h2o


In [13]:
output_config_filename = os.path.join(config_output_directory, os.path.basename(source_config_filename))

if not os.path.exists(config_output_directory):
    os.makedirs(config_output_directory)

with open(output_config_filename, "w") as out_file:
    config.write(out_file)
    
print("Wrote new config file:", output_config_filename)

Wrote new config file: /tmp/absco_build/tropomi/ABSCO_config.ini
